### making sentiment analyzer for email classification

In [1]:
import time
import requests
import pandas as pd
from tqdm import tqdm
from functools import reduce
from bs4 import BeautifulSoup
import undetected_chromedriver as uc
from linkedin_scraper import actions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

def Indeed_scrape():
    global df_main_indeed
    global s

    url = "https://in.indeed.com/companies?from=gnav-acme--acme-webapp"
    driver.get(url)
    main_page = driver.current_window_handle

    #clicking the sign-in button
    driver.find_element(By.XPATH, '//*[@id="gnav-main-container"]/div/div/div[2]/div[2]/div[2]/a').click()

    #clicking on sign-in using google button
    driver.find_element(By.XPATH, '//*[@id="login-google-button"]').click()

    # changing windows
    for handle in driver.window_handles:
        if handle != main_page:
            login_page = handle
    driver.switch_to.window(login_page)

    
    try:
        enter_email = driver.find_element(By.XPATH, '//*[@id="identifierId"]')
        enter_email.send_keys(email)
        enter_email.send_keys(Keys.RETURN)
        time.sleep(5)
        enter_pass = driver.find_element(By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')
        enter_pass.send_keys(p)
        driver.switch_to.window(login_page)
        enter_pass.send_keys(Keys.RETURN)
    
    except:
        if "Use another account" in driver.page_source:
            driver.find_element(By.XPATH, '//*[@id="view_container"]/div/div/div[2]/div/div[1]/div/form/span/section/div/div/div/div/ul/li[1]/div/div[1]/div/div[2]/div[1]').click()
            time.sleep(3)

    driver.switch_to.window(main_page)
    time.sleep(3)

    if s == 'CSL Finance' or s == 'csl finance' or s == 'CSL FINANCE':
        s = 'CSL Finance Limited'

    #searching for the company
    wait = WebDriverWait(driver, 500)
    text_element = wait.until(EC.visibility_of_element_located((By.XPATH, "//h1[contains(text(), 'Find great places to work')]")))
    driver.get("https://in.indeed.com/cmp/{}/reviews".format(s))

    soup = BeautifulSoup(driver.page_source, 'lxml')

    if "Be the first to review this employer." in driver.page_source:
        print('No data available on Indeed')
        df_main_indeed = pd.DataFrame()

    soup = BeautifulSoup(driver.page_source, 'lxml')
    if " reviews matching the search" in driver.page_source:
        soup = BeautifulSoup(driver.page_source, 'lxml')
        if "We can't find this page" in driver.page_source:
            print('No data found on Indeed')
            df_main_indeed = pd.DataFrame
        else:
            #sign in the browser first
            #using BeautifulSoup4 to scrape out page source of the webpage
            while True:
                # Search for "Review this company"" on the webpage
                if "Review this company" in driver.page_source:
                    data = requests.get(url)
                    soup = BeautifulSoup(driver.page_source, 'lxml')
                    # print(soup)
                    break  # Exit the loop if the text is found
                



            def complete_reviews(soup):
                #extracting all data from customer-reviews section

                #review_text of review
                reviews_text = []
                temp6 = soup.find_all('span', attrs={"css-1cxc9zk e1wnkr790"})
                for i in temp6:
                    reviews_text.append(i.text)
                reviews_text = reviews_text[1:]

                star_ratings = []
                star = soup.find_all('button', attrs={'class': 'css-1c33izo e1wnkr790'})
                for i in star:
                    star_ratings.append(i.text)

                #position of each customer
                pos_of_customer = []
                for pos in soup.find_all('div', attrs={'class': 'css-lw17hn eu4oa1w0'}):
                    pos1 = pos.find('a', attrs={'class': 'css-rt1o3i e19afand0'})
                    if pos1:
                        pos_of_customer.append(pos1.text)
                    else:
                        pos_of_customer.append('NA')

                #complete line containing position, date etc. to extract out date and location
                complete_line = []
                for cl in soup.find_all('div', attrs={'class': 'css-lw17hn eu4oa1w0'}):
                    cl1 = cl.find('span', attrs={'itemprop': 'author'})
                    if cl1:
                        complete_line.append(cl1.text)
                    else:
                        complete_line.append('NA')

                #extracting date from complete_line list
                #splitting by (-)
                date_of_review = []
                for i in complete_line:
                    date_of_review.append(i.split('- ')[2])

                #extracting employee status from complete_line list
                #splitting by ('(') and then by (')')
                employee_status = []
                for item in complete_line:
                    inner_list = []
                    for subitem in item.split('('):
                        inner_list.extend(subitem.split(')'))
                    employee_status.append(inner_list[1])

                
                # Ensure all arrays have the same length
                min_length = min(len(reviews_text), len(star_ratings), len(pos_of_customer), len(date_of_review), len(employee_status))
                reviews_text = reviews_text[:min_length]
                star_ratings = star_ratings[:min_length]
                pos_of_customer = pos_of_customer[:min_length]
                date_of_review = date_of_review[:min_length]
                employee_status = employee_status[:min_length]

                all_pages = {}
                all_pages['Reviews'] = reviews_text
                all_pages['Rating'] = star_ratings
                all_pages['Designation'] = pos_of_customer
                all_pages['Date'] = date_of_review
                all_pages['Employee Status'] = employee_status
                return all_pages

            soup = BeautifulSoup(driver.page_source, 'lxml')
            if "Showing the only review" in driver.page_source:
                soup = BeautifulSoup(driver.page_source, 'lxml')
                #extracting all data from customer-reviews section

                #review_text of review
                reviews_text = []
                temp6 = soup.find_all('span', attrs={"css-1cxc9zk e1wnkr790"})
                for i in temp6:
                    reviews_text.append(i.text)
                reviews_text = reviews_text[1:]

                star_ratings = []
                star = soup.find_all('button', attrs={'class': 'css-1c33izo e1wnkr790'})
                for i in star:
                    star_ratings.append(i.text)

                #position of each customer
                pos_of_customer = []
                for pos in soup.find_all('div', attrs={'class': 'css-lw17hn eu4oa1w0'}):
                    pos1 = pos.find('a', attrs={'class': 'css-rt1o3i e19afand0'})
                    if pos1:
                        pos_of_customer.append(pos1.text)
                    else:
                        pos_of_customer.append('NA')

                #complete line containing position, date etc. to extract out date and location
                complete_line = []
                for cl in soup.find_all('div', attrs={'class': 'css-lw17hn eu4oa1w0'}):
                    cl1 = cl.find('span', attrs={'itemprop': 'author'})
                    if cl1:
                        complete_line.append(cl1.text)
                    else:
                        complete_line.append('NA')

                #extracting date from complete_line list
                #splitting by (-)
                date_of_review = []
                for i in complete_line:
                    date_of_review.append(i.split('- ')[2])

                #extracting employee status from complete_line list
                #splitting by ('(') and then by (')')
                employee_status = []
                for item in complete_line:
                    inner_list = []
                    for subitem in item.split('('):
                        inner_list.extend(subitem.split(')'))
                    employee_status.append(inner_list[1])

                
                # Ensure all arrays have the same length
                min_length = min(len(reviews_text), len(star_ratings), len(pos_of_customer), len(date_of_review), len(employee_status))
                reviews_text = reviews_text[:min_length]
                star_ratings = star_ratings[:min_length]
                pos_of_customer = pos_of_customer[:min_length]
                date_of_review = date_of_review[:min_length]
                employee_status = employee_status[:min_length]

                all_pages = {}
                all_pages['Reviews'] = reviews_text
                all_pages['Rating'] = star_ratings
                all_pages['Designation'] = pos_of_customer
                all_pages['Date'] = date_of_review
                all_pages['Employee Status'] = employee_status
                df_main_indeed = pd.DataFrame(all_pages)
                df_main_indeed["Source"] = "Indeed"
                print("\nIndeed's Data Extraction | Completed\n")

            else:
                text1 = driver.find_element(By.XPATH, '//*[@id="cmp-container"]/div/div[1]/main/div[2]/div[1]/div[1]/div[1]/span/b')
                text1_content = text1.text.replace(',', '')
                int_text1 = int(text1_content)
                no_of_pages = int(int_text1/21)
                end_val = (no_of_pages)*20
                df_main_indeed = pd.DataFrame()
                for i in tqdm(range(20, end_val+1, 20)):
                    try:
                        wait = WebDriverWait(driver, 100)
                        try:
                            wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".css-13qwiat.e71d0lh0[data-tn-element='next-page']"))).click()
                        except NoSuchElementException:
                            # Element not found, break the loop
                            break
                        time.sleep(4)
                        soup = BeautifulSoup(driver.page_source, 'lxml')
                        all_reviews = complete_reviews(soup)
                        df = pd.DataFrame(all_reviews)
                        df_main_indeed = pd.concat([df_main_indeed, df], ignore_index=True)
                        df_main_indeed = df_main_indeed.reset_index(drop=True)
                    except:
                        print(i)
                    df_main_indeed["Source"] = "Indeed"
                print("\nIndeed's Data Extraction | Completed\n")
        
def Twitter_scrape():
    driver.get('https://twitter.com/')
    main_page = driver.current_window_handle
    time.sleep(5)

    #Login procedure
    driver.switch_to.frame(0)
    login = driver.find_element('xpath','//*[@id="container"]')
    ActionChains(driver).move_to_element(login).click().perform()
    login.click()
    for handle in driver.window_handles:
        if handle != main_page:
            login_page = handle
    driver.switch_to.window(login_page)
    try:
        log = driver.find_element('xpath', '//*[@id="identifierId"]')
        log.send_keys(email)
        log.send_keys(Keys.RETURN)
        driver.implicitly_wait(10)
        pas =  driver.find_element('xpath', '//*[@id="password"]/div[1]/div/div[1]/input')
        pas.send_keys(p)
        pas.send_keys(Keys.RETURN)
        time.sleep(5)
    except:
        log = driver.find_element('xpath','//*[@role="link"]')
        log.click()
    driver.switch_to.window(main_page)
    time.sleep(5)

    #Searching for the organisation
    search = driver.find_element('xpath','//input[@placeholder="Search Twitter"]')
    search.click()
    search.send_keys(f"{s}")
    search.send_keys(Keys.RETURN)
    time.sleep(3)

    #Scraper
    def get_data(tweet):
        try:
            postdate = tweet.find_element('xpath','.//time').get_attribute('datetime')
        except NoSuchElementException:
            return
        try:
            post = tweet.find_element('xpath','.//div[@data-testid="tweetText"]').text
        except NoSuchElementException:
            return
        reply = tweet.find_element('xpath','//div[@data-testid="reply"]').text
        retweet = tweet.find_element('xpath','.//div[@data-testid="retweet"]').text
        like = tweet.find_element('xpath','.//div[@data-testid="like"]').text
        views = tweet.find_element('xpath','.//div[@role="group"]/div[4]').text
        
        dets = (postdate, post, reply, retweet, like, views)
        return dets

    data = []
    tweet_ids = set()
    page_end = False
    height = driver.execute_script("return window.pageYOffset;")
    while page_end !=True:
        tweets = driver.find_elements('xpath','//article[@data-testid="tweet"]')
        for tweet in tweets:
            dat = get_data(tweet)
            if dat:
                tweet_id = ''.join(dat)
                if tweet_id not in tweet_ids:
                    tweet_ids.add(tweet_id)
                    data.append(dat)
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(1)
        new_height = driver.execute_script("return window.pageYOffset;")
        if height == new_height:
            page_end = True
        else:
            height = new_height
    
    T = pd.DataFrame(data,columns =['Posted on', 'Tweet', 'Reply', 'Retweets', 'Likes', 'Views'])
    T = T.rename(columns = {'Tweet':'Reviews'})
    T['Date']= pd.to_datetime(T['Posted on'], errors = 'coerce')
    T['Date'] = T['Date'].dt.date
    T = T[['Date','Reviews']]
    T['Source'] = 'Twitter'
    print("'\nTwitter Data Extraction | Completed\n'")
    return T

def AmbitionBox_scrape():
    driver.get("https://www.ambitionbox.com/")
    main_page = driver.current_window_handle
    time.sleep(2)

    #Login Procedure
    login = driver.find_element('xpath','//*[@id="ambitionbox-header"]/div/div[2]/div/div[4]/a')
    ActionChains(driver).move_to_element(login).click().perform()
    time.sleep(1)
    g_log = driver.find_element('xpath','//*[@class="social-login__button google"]')
    ActionChains(driver).move_to_element(g_log).click().perform()
    time.sleep(1)

    for handle in driver.window_handles:
        if handle != main_page:
            login_page = handle
    driver.switch_to.window(login_page)
    try:
        login = driver.find_element('xpath', '//*[@id="identifierId"]')
        login.send_keys(email)
        login.send_keys(Keys.RETURN)
        driver.implicitly_wait(10)
        pas =  driver.find_element('xpath', '//*[@id="password"]/div[1]/div/div[1]/input')
        pas.send_keys(p)
        pas.send_keys(Keys.RETURN)
        time.sleep(3)
    except:
        if "Use another account" in driver.page_source:
            driver.find_element(By.XPATH, '//*[@id="view_container"]/div/div/div[2]/div/div[1]/div/form/span/section/div/div/div/div/ul/li[1]/div/div[1]/div/div[2]/div[1]').click()
            time.sleep(3)


    driver.switch_to.window(main_page)
    time.sleep(3)

    #Searching for the organisation
    search = driver.find_element('xpath','//*[@id="homeTypeahead"]')
    search.send_keys(s)
    search.send_keys(Keys.RETURN)
    time.sleep(3)

    current_url = driver.current_url
    current_url = str(current_url)
    filter_text = '?sort_by=latest'
    filter_url = current_url + filter_text
    driver.get(filter_url)
    time.sleep(5)

    def rev_scrape(driver):
        soup = BeautifulSoup(driver.page_source,'lxml')
        like = []
        dislike = []
        rev_date = []
        ind_rate = []
        des = []
        review = {}
        rev = soup.find_all('p', attrs = {'class': 'body-medium overflow-wrap'})
        r_date = soup.find_all('span', attrs = {'class': 'status caption-subdued'})
        r = soup.find_all('span', attrs = {'class': 'avg-rating bold-Labels'})
        d = soup.find_all('h2', attrs = {'class': 'bold-title-s review-title'})

        for i in range(len(rev)):
            if i % 2 == 0:
                like.append(rev[i].text)
            else:
                dislike.append(rev[i].text)
        for i in range(len(r_date)):
            rev_date.append(r_date[i].text.replace('posted on', ''))
            ind_rate.append(r[i].text)
            des.append(d[i].text)
        
        review["Designation"] = des
        review["Rating"] = ind_rate
        review["Date of Review"] = rev_date
        review["Like"] = like
        review["Dislike"] = dislike
        return review
    
    i = True
    AB=pd.DataFrame()
    while(i):
        try:
            dict = rev_scrape(driver)
            df = pd.DataFrame.from_dict(dict,orient='index').transpose()
            AB = pd.concat([AB, df], ignore_index=True)
            button_l = driver.find_element('xpath','//*[@class="icon-chevron-right next nav-btn"]')
            button_l.click()
            time.sleep(3)
        except:
            i= False
    
    AB['Reviews'] = AB['Like'] + "  " + AB['Dislike']
    AB = AB.rename(columns = {'Date of Review': 'Date'})
    AB['Date'] = pd.to_datetime(AB['Date'])
    AB['Date'] = AB['Date'].dt.date
    AB = AB[['Date','Reviews','Rating']]
    AB['Source'] = 'Ambition Box'
    print("\Ambition Box's Data Extraction | Completed\n")
    return AB

def Facebook_scrape():
    driver.get("https://www.facebook.com")
    driver.find_element(By.XPATH, "//*[@id='email']").send_keys(email)              #entering email-id
    driver.find_element(By.XPATH, "//*[@id='email']").send_keys(Keys.TAB)
    driver.find_element(By.XPATH, "//*[@id='pass']").send_keys(p)               #entering password
    driver.find_element(By.XPATH, "//*[@id='pass']").send_keys(Keys.RETURN)            #hitting enter
    time.sleep(5)
    # driver.find_element(By.CLASS_NAME, 'x1uvtmcs x4k7w5x x1h91t0o x1beo9mf xaigb6o x12ejxvf x3igimt xarpa2k xedcshv x1lytzrv x1t2pt76 x7ja8zs x1n2onr6 x1qrby5j x1jfb8zj').send_keys(Keys.ESCAPE)
    #going to companies post webpage
    if 'Find friends' in driver.page_source:
        driver.get(f'https://www.facebook.com/search/posts?q={s}&filters=eyJyZWNlbnRfcG9zdHM6MCI6IntcIm5hbWVcIjpcInJlY2VudF9wb3N0c1wiLFwiYXJnc1wiOlwiXCJ9In0%3D')
        time.sleep(3)

    # Function to scroll to the bottom of the page (page 2023)
    def scroll_to_bottom():
        SCROLL_PAUSE_TIME = 2  # Adjust the pause time if needed  
        while True:
            # Scroll to the bottom of the page
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            # Wait for the new content to load
            WebDriverWait(driver, SCROLL_PAUSE_TIME).until(EC.invisibility_of_element_located((By.CLASS_NAME, "uiMorePagerLoader")))
            if driver.find_elements(By.XPATH, "//span[contains(text(), 'End of results')]"):
                break  # Reached the bottom of the page                                                                                                     
    scroll_to_bottom()

    #using BeautifulSoup4 to scrape out data from the posts
    soup = BeautifulSoup(driver.page_source, 'lxml')
    #scraping data from the posts from filter of latest reviews
    #extracting text from the posts
    text_list = []
    for tl in soup.find_all('div', attrs={'class': 'x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z'}):
        tl1 = tl.find('span',attrs={'class': 'x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x3x7a5m x6prxxf xvq8zen xo1l8bm xzsf02u x1yc453h'})
        if tl1:
            text_list.append(tl1.text)
    #making a dictionary containing all the list to make the final dataframe
    complete_list = {"Reviews":text_list}

    #creating final dataframe
    df_main = pd.DataFrame(complete_list)
    df_main["Source"] = "Facebook"
    print("\nFacebook's Data Extraction | Completed\n")
    return df_main

def TOI_scrape():                                            #opening browser
    driver.get('https://timesofindia.indiatimes.com/advancesearch.cms')         #going to the video
    driver.maximize_window()
    time.sleep(1)

    input_button = driver.find_element(By.XPATH, '//*[@id="frm1"]/div[3]/input')
    ActionChains(driver).move_to_element(input_button).click().send_keys(s).perform()

    #setting yearly filter
    driver.find_element(By.XPATH, '//*[@id="frm1"]/div[7]/span[1]/input').click()

    #clicking submit button
    driver.find_element(By.XPATH, '//*[@id="frm1"]/div[9]/div').click()
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'lxml')
    if " - did not match any documents." in driver.page_source:
        print('No data found on TOI')
        df_main_toi = pd.DataFrame
    else:
        no_of_articles = driver.find_element(By.XPATH, '//*[@id="netspidersosh"]/div[1]/div[2]/div/div[2]/div[2]/div[2]/span[3]')
        no_of_articles = int(no_of_articles.text)

        if no_of_articles<=10:
            soup = BeautifulSoup(driver.page_source, 'lxml')
            
            #extracting reviews
            reviews = []
            for tl in soup.find_all('div', attrs={'style': 'padding-bottom:20px;font-size:13px;'}):
                tl1 = tl.find('span', attrs={'style': 'font-family:arial'})
                if tl1:
                    reviews.append(tl1.text)
            reviews = list(set(reviews))  

            all_pages = {}
            all_pages['Reviews'] = reviews
            df_main_toi = pd.DataFrame(all_pages)
            df_main_toi["Source"] = "Times of India"
            df_main_toi.dropna(subset=["Reviews"], inplace=True)
            print("\nTOI's Data Extraction | Completed\n")
            return df_main_toi

        if no_of_articles>10:
            no_of_scrolls = int(no_of_articles/10)
            def complete_reviews(soup):
                soup = BeautifulSoup(driver.page_source, 'lxml')
                #extracting reviews
                reviews = []
                for tl in soup.find_all('div', attrs={'style': 'padding-bottom:20px;font-size:13px;'}):
                    tl1 = tl.find('span', attrs={'style': 'font-family:arial'})
                    if tl1:
                        reviews.append(tl1.text)
                reviews = list(set(reviews))  

                all_pages = {}
                all_pages['Reviews'] = reviews
                return all_pages

            df_main_toi=pd.DataFrame()
            for i in tqdm(range(1,no_of_scrolls+1,1)):
                try:
                    url = f"https://timesofindia.indiatimes.com/searchresult.cms?maxRow=&kdaterange=365&article=2&searchtype=2&section=&searchField=&query={s}&pagenumber={i}&startdate=2001-01-01&enddate=2001-07-26&date1mm=01&date1dd=01&date1yyyy=2001&date2mm=07&date2dd=26&date2yyyy=2001"
                    driver.get(url)
                    soup  = BeautifulSoup(driver.page_source,'lxml')
                    dict = complete_reviews(soup)
                    df=pd.DataFrame.from_dict(dict,orient='index').transpose()
                    df_main_toi = pd.concat([df_main_toi, df], ignore_index=True)
                except:
                    print(i)
            df_main_toi["Source"] = "Times of India"
            df_main_toi.dropna(subset=["Reviews"], inplace=True)
            print("\nTOI's Data Extraction | Completed\n")
            return df_main_toi
    
def LinkedIn_scrape():
    driver.get('https://www.linkedin.com')
    actions.login(driver, email2, p2)

    # Searching for the organisation
    search = driver.find_element('xpath','//*[@id="global-nav-typeahead"]/input')
    search.send_keys(s)
    search.send_keys(Keys.RETURN)
    time.sleep(5)

    # Check for any known company
    fill = driver.find_elements('xpath','//*[@class = "search-reusables__primary-filter"]')
    for i in fill:
        if i.text == 'Companies':
            i.click()
            break
    else:
        print("No such organisation is on LinkedIn.")
    try:
        if "No results for" in driver.find_element('xpath','//*[@class = "t-14 t-black--light"]').text:
                print("No results found for the searched entity")
    except:
        pass
    time.sleep(5)
    # First search result
    comp = driver.find_elements('xpath','//*[@class="reusable-search__result-container"]')
    for i in range(len(comp)):
         if 'Follow' in comp[i].text:
                comp[i].click()
                break
         else:
                pass
    time.sleep(5)

    # Post's Tab
    try:
        for i in range(1,6):
            tab = driver.find_element('xpath',f'//*[@aria-label="Organization’s page navigation"]/ul/li[{i}]')
            if tab.text == "Posts":
                tab.click()
    except:
        print("No Post's Tab found!")

    def scrolling(driver):
        page_end = False
        height = 0
        while page_end !=True:
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            new_height = driver.execute_script("return document.body.scrollHeight")
            time.sleep(3)
            if height == new_height:
                page_end = True
            else:
                height = new_height
    
    scrolling(driver)
    posts = []
    dets = []
    soup = BeautifulSoup(driver.page_source,'lxml')
    post = driver.find_elements('xpath', '//div[@class="ember-view  occludable-update "]')
    det = soup.find_all('ul', attrs = {'class': 'social-details-social-counts'})

    for i in range(len(post)):
        posts.append(post[i].text.replace('\n', ''))
        dets.append(det[i].text.replace('\n', ''))

    Post = []
    import re
    for i in range(len(posts)):
        temp = re.split('•Follow|…see more',posts[i])
        Post.append(temp[1])
    like = []
    comment = [None] * len(dets)
    repost = [None] * len(dets)
    for i in range(len(dets)):
        temp = dets[i].split("  ")
        like.append(temp[0])
        for j in range(len(temp)):
            if 'comments' in temp[j]:
                comment[i] = temp[j]
            if 'reposts' in temp[j]:
                repost[i] = temp[j]
    
    L = pd.DataFrame(list(zip(Post, like,comment,repost)), columns =['Post', 'Likes','Comments','Reposts'])
    L['Date'] = None
    L = L.rename(columns = {'Post':'Reviews'})
    L = L[['Date','Reviews']]
    L['Source'] = 'LinkedIn'
    print("\nLinkedIn's Data Extraction | Completed\n")
    return L

def Glassdoor_scrape():
    #using selenium for automation
    url = "https://www.glassdoor.co.in/member/home/index.htm"
    driver.get(url)
    driver.maximize_window()
    driver.back()
    driver.forward()
    global s
    #logging-in
    wait = WebDriverWait(driver, 1000)
    if wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="InlineLoginModule"]/div/div/div[1]/div/div/div/div/form/div[1]/label'))):
        #entering email
        enter_email = driver.find_element(By.XPATH, '//*[@id="inlineUserEmail"]')
        ActionChains(driver).move_to_element(enter_email).click().send_keys(email).send_keys(Keys.RETURN).perform()
    else:
        time.sleep(2)
        #entering password
    wait = WebDriverWait(driver, 10)
    if wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="InlineLoginModule"]/div/div/div[1]/div/div/div/div/button'))):
        enter_password = driver.find_element(By.XPATH, '//*[@id="inlineUserPassword"]')
        ActionChains(driver).move_to_element(enter_password).click().send_keys(p).send_keys(Keys.RETURN).perform()

    time.sleep(2)
    driver.get(url)                 #sometimes, some alternate page opens up, in that case, this line is required

    if searches == 'rigi' or searches == 'Rigi' or searches == 'RIGI':
        s = 'Rigi (India)'

    #waiting for page to load
    time.sleep(3)
    driver.get('https://www.glassdoor.co.in/Reviews/index.htm?overall_rating_low=3.5&page=1&locId=2921225&locType=C&occ=Data%20Analyst')

    #entering comapany's name in search box 
    wait = WebDriverWait(driver, 100)
    if wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="Explore"]/div[2]/div/div/div[1]'))):
        enter_company = driver.find_element(By.XPATH, '//*[@id="companyAutocomplete-companyDiscover-employerSearch"]')
        ActionChains(driver).move_to_element(enter_company).click().send_keys(s).perform()
    else:
        time.sleep(2)

    driver.find_element(By.XPATH, '//*[@id="Explore"]/div[2]/div/div/div[2]/button').click()

    soup = BeautifulSoup(driver.page_source, 'lxml')
    if " Showing results for " in driver.page_source:
        soup = BeautifulSoup(driver.page_source, 'lxml')
        if "Check your spelling" in driver.page_source:
            print('No data found on Glassdoor')
            global df_main_glassdoor
            df_main_glassdoor = pd.DataFrame
        else:
            time.sleep(5)
            driver.find_element(By.XPATH, '//*[@id="MainCol"]/div/div[1]/div/div[1]/div/div[2]/h2/a').click()

            time.sleep(5) 
            driver.find_element(By.XPATH, '//*[@id="EmpLinksWrapper"]/div[2]/div/div[1]/a[1]').click()

            #chosing filter
            time.sleep(3)
            driver.find_element(By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[1]/div[1]/div/div[1]/div[2]/div[2]/button').click()
            time.sleep(2)
            loc_filter = driver.find_element(By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[1]/div[1]/div/div[1]/div[3]/div[2]/div[1]/div/div[1]')
            ActionChains(driver).move_to_element(loc_filter).click().click().send_keys('India ').send_keys(Keys.RETURN).perform()

            #waiting for filtered-review page to load and then finding soup
            wait = WebDriverWait(driver, 100)
            if wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[1]/div[3]/div[1]/div/span'))):
                soup = BeautifulSoup(driver.page_source, 'lxml')
            else:
                time.sleep(2)
            #extracting all data from customer-reviews section in a function 


            def complete_reviews(soup):

                #extracting the pro reviews from all the reviews
                pros_reviews = []
                for pros in soup.find_all('div', attrs={'class': 'gdReview'}):
                    pros1 = pros.find('span', attrs={'data-test': "pros"})
                    if pros1:
                        pros_reviews.append(pros1.text)
                    else:
                        pros_reviews.append('NA')

                #extracting the cons reviews from all the reviews
                cons_reviews = []
                for cons in soup.find_all('div', attrs={'class': 'gdReview'}):
                    cons1 = cons.find('span', attrs={'data-test': "cons"})
                    if cons1:
                        cons_reviews.append(cons1.text)
                    else:
                        cons_reviews.append('NA')

                #saving the pros and cons in one column named "Reviews"
                combined_reviews = list(zip(pros_reviews, cons_reviews))

                #extracting the star ratings from the reviews
                star_ratings = []
                for star in soup.find_all('div', attrs={'class': 'gdReview'}):
                    star1 = star.find('span', attrs={'class': 'ratingNumber mr-xsm'})
                    if star1:
                        star_ratings.append(star1.text)
                    else:
                        star_ratings.append('NA')

                #extracting the employee status line to extract employee status from it
                emp_line = []
                for empl in soup.find_all('div', attrs={'class': 'gdReview'}):
                    empl1 = empl.find('span', attrs={'class': 'pt-xsm pt-md-0 css-1qxtz39 eg4psks0'})
                    if empl1:
                        emp_line.append(empl1.text)
                    else:
                        emp_line.append('NA')
                
                # extracting status of employee
                employee_status = []
                for i in emp_line:
                    employee_status.append(i.split(',')[0])
                
                #extracting the line that contains the date and designation in the review
                date_and_des = []
                for ded in soup.find_all('div', attrs={'class': 'gdReview'}):
                    ded1 = ded.find('span', attrs={'class': 'middle common__EiReviewDetailsStyle__newGrey'})
                    if ded1:
                        date_and_des.append(ded1.text)
                    else:
                        date_and_des.append('NA')

                #extracting the date from the date_and_des list
                date_of_review = []
                for i in date_and_des:
                    date_of_review.append(i.split(' -')[0])

                #extracting the designation from the date_and_des list
                designation_of_reviewer = []
                for i in date_and_des:
                    designation_of_reviewer.append(i.split('- ')[-1])

                all_pages = {}
                all_pages['Reviews'] = combined_reviews
                all_pages['Rating'] = star_ratings
                all_pages['Designation'] = designation_of_reviewer
                all_pages['Date'] = date_of_review
                all_pages['Employee Status'] = employee_status

                return all_pages

            #counting the number of pages to be paginated
            total_reviews = driver.find_element(By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[1]/div[3]/div[1]/h2/span/strong[1]')
            total_reviews = int(total_reviews.text.replace(',',''))       #total number of reviews after filter
            no_of_pages = int(total_reviews/10)                                #number of pages to be scrolled
            
            if no_of_pages<1:
                soup = BeautifulSoup(driver.page_source, 'lxml')
                #extracting all data from customer-reviews section in a function 

                #extracting the pro reviews from all the reviews
                pros_reviews = []
                for pros in soup.find_all('div', attrs={'class': 'gdReview'}):
                    pros1 = pros.find('span', attrs={'data-test': "pros"})
                    if pros1:
                        pros_reviews.append(pros1.text)
                    else:
                        pros_reviews.append('NA')

                #extracting the cons reviews from all the reviews
                cons_reviews = []
                for cons in soup.find_all('div', attrs={'class': 'gdReview'}):
                    cons1 = cons.find('span', attrs={'data-test': "cons"})
                    if cons1:
                        cons_reviews.append(cons1.text)
                    else:
                        cons_reviews.append('NA')

                #saving the pros and cons in one column named "Reviews"
                combined_reviews = list(zip(pros_reviews, cons_reviews))

                #extracting the star ratings from the reviews
                star_ratings = []
                for star in soup.find_all('div', attrs={'class': 'gdReview'}):
                    star1 = star.find('span', attrs={'class': 'ratingNumber mr-xsm'})
                    if star1:
                        star_ratings.append(star1.text)
                    else:
                        star_ratings.append('NA')

                #extracting the employee status line to extract employee status from it
                emp_line = []
                for empl in soup.find_all('div', attrs={'class': 'gdReview'}):
                    empl1 = empl.find('span', attrs={'class': 'pt-xsm pt-md-0 css-1qxtz39 eg4psks0'})
                    if empl1:
                        emp_line.append(empl1.text)
                    else:
                        emp_line.append('NA')

                # extracting status of employee
                employee_status = []
                for i in emp_line:
                    employee_status.append(i.split(',')[0])
                
                #extracting the line that contains the date and designation in the review
                date_and_des = []
                for ded in soup.find_all('div', attrs={'class': 'gdReview'}):
                    ded1 = ded.find('span', attrs={'class': 'middle common__EiReviewDetailsStyle__newGrey'})
                    if ded1:
                        date_and_des.append(ded1.text)
                    else:
                        date_and_des.append('NA')

                #extracting the date from the date_and_des list
                date_of_review = []
                for i in date_and_des:
                    date_of_review.append(i.split(' -')[0])

                #extracting the designation from the date_and_des list
                designation_of_reviewer = []
                for i in date_and_des:
                    designation_of_reviewer.append(i.split('- ')[-1])

                all_pages = {}
                all_pages['Reviews'] = combined_reviews
                all_pages['Rating'] = star_ratings
                all_pages['Designation'] = designation_of_reviewer
                all_pages['Date'] = date_of_review
                all_pages['Employee Status'] = employee_status

                df_main_glassdoor = pd.DataFrame(all_pages)
                df_main_glassdoor["Source"] = "Glassdoor"
                print("\nGlassdoor's Data Extraction | Completed\n")
            
            if no_of_pages >= 1:
                # Creating final dataframe & pagination using link
                df_main_glassdoor = pd.DataFrame()
                wait = WebDriverWait(driver, 100)
                try:
                    wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[4]/div/div[1]/button[2]'))).click()
                except NoSuchElementException:
                    # If the first XPath is not found, try the second one
                    wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[6]/div/div[1]/button[2]'))).click()
                for i in tqdm(range(1, no_of_pages + 1, 1)):
                    wait = WebDriverWait(driver, 100)
                    try:
                        # try:
                        #     wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[6]/div/div[1]/button[2]'))).click()
                        # except NoSuchElementException:
                        #     # If the first XPath is not found, try the second one
                        wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[6]/div/div[1]/button[2]'))).click()
                    
                        time.sleep(4)
                        # Scroll to the next button element
                        next_button = driver.find_element(By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[6]/div/div[1]/button[2]')
                        actions = ActionChains(driver)
                        actions.move_to_element(next_button).perform()
                        
                        soup = BeautifulSoup(driver.page_source, 'lxml')
                        all_reviews = complete_reviews(soup)
                        df = pd.DataFrame(all_reviews)
                        df_main_glassdoor = pd.concat([df_main_glassdoor, df], ignore_index=True)
                        df_main_glassdoor = df_main_glassdoor.reset_index(drop=True)
                    except:
                        print(i)



                df_main_glassdoor["Source"] = "Glassdoor"
                print("\nGlassdoor's Data Extraction | Completed\n")
    else:
        time.sleep(4) 
        driver.find_element(By.XPATH, '//*[@id="EmpLinksWrapper"]/div[2]/div/div[1]/a[1]').click()

        #chosing filter
        wait = WebDriverWait(driver, 100)
        if wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[1]/div[3]/div[1]/h2'))):
            driver.find_element(By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[1]/div[1]/div/div[1]/div[2]/div[2]/button').click()
            time.sleep(2)
            loc_filter = driver.find_element(By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[1]/div[1]/div/div[1]/div[3]/div[2]/div[1]/div/div[1]')
            ActionChains(driver).move_to_element(loc_filter).click().click().send_keys('India ').send_keys(Keys.RETURN).perform()
        else:
            time.sleep(2)

        #waiting for filtered-review page to load and then finding soup
        wait = WebDriverWait(driver, 100)
        if wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[1]/div[3]/div[1]/div/span'))):
            soup = BeautifulSoup(driver.page_source, 'lxml')
        else:
            time.sleep(2)
        #extracting all data from customer-reviews section in a function 


        def complete_reviews(soup):

            #extracting the pro reviews from all the reviews
            pros_reviews = []
            for pros in soup.find_all('div', attrs={'class': 'gdReview'}):
                pros1 = pros.find('span', attrs={'data-test': "pros"})
                if pros1:
                    pros_reviews.append(pros1.text)
                else:
                    pros_reviews.append('NA')

            #extracting the cons reviews from all the reviews
            cons_reviews = []
            for cons in soup.find_all('div', attrs={'class': 'gdReview'}):
                cons1 = cons.find('span', attrs={'data-test': "cons"})
                if cons1:
                    cons_reviews.append(cons1.text)
                else:
                    cons_reviews.append('NA')

            #saving the pros and cons in one column named "Reviews"
            combined_reviews = list(zip(pros_reviews, cons_reviews))

            #extracting the star ratings from the reviews
            star_ratings = []
            for star in soup.find_all('div', attrs={'class': 'gdReview'}):
                star1 = star.find('span', attrs={'class': 'ratingNumber mr-xsm'})
                if star1:
                    star_ratings.append(star1.text)
                else:
                    star_ratings.append('NA')

            #extracting the employee status line to extract employee status from it
            emp_line = []
            for empl in soup.find_all('div', attrs={'class': 'gdReview'}):
                empl1 = empl.find('span', attrs={'class': 'pt-xsm pt-md-0 css-1qxtz39 eg4psks0'})
                if empl1:
                    emp_line.append(empl1.text)
                else:
                    emp_line.append('NA')
            
            # extracting status of employee
            employee_status = []
            for i in emp_line:
                employee_status.append(i.split(',')[0])
            
            #extracting the line that contains the date and designation in the review
            date_and_des = []
            for ded in soup.find_all('div', attrs={'class': 'gdReview'}):
                ded1 = ded.find('span', attrs={'class': 'middle common__EiReviewDetailsStyle__newGrey'})
                if ded1:
                    date_and_des.append(ded1.text)
                else:
                    date_and_des.append('NA')

            #extracting the date from the date_and_des list
            date_of_review = []
            for i in date_and_des:
                date_of_review.append(i.split(' -')[0])

            #extracting the designation from the date_and_des list
            designation_of_reviewer = []
            for i in date_and_des:
                designation_of_reviewer.append(i.split('- ')[-1])

            all_pages = {}
            all_pages['Reviews'] = combined_reviews
            all_pages['Rating'] = star_ratings
            all_pages['Designation'] = designation_of_reviewer
            all_pages['Date'] = date_of_review
            all_pages['Employee Status'] = employee_status

            return all_pages

        #counting the number of pages to be paginated
        total_reviews = driver.find_element(By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[1]/div[3]/div[1]/h2/span/strong[1]')
        total_reviews = int(total_reviews.text.replace(',',''))       #total number of reviews after filter
        no_of_pages = int(total_reviews/10)                                #number of pages to be scrolled
        
        if no_of_pages<1:
            soup = BeautifulSoup(driver.page_source, 'lxml')
            #extracting all data from customer-reviews section in a function 

            #extracting the pro reviews from all the reviews
            pros_reviews = []
            for pros in soup.find_all('div', attrs={'class': 'gdReview'}):
                pros1 = pros.find('span', attrs={'data-test': "pros"})
                if pros1:
                    pros_reviews.append(pros1.text)
                else:
                    pros_reviews.append('NA')

            #extracting the cons reviews from all the reviews
            cons_reviews = []
            for cons in soup.find_all('div', attrs={'class': 'gdReview'}):
                cons1 = cons.find('span', attrs={'data-test': "cons"})
                if cons1:
                    cons_reviews.append(cons1.text)
                else:
                    cons_reviews.append('NA')

            #saving the pros and cons in one column named "Reviews"
            combined_reviews = list(zip(pros_reviews, cons_reviews))

            #extracting the star ratings from the reviews
            star_ratings = []
            for star in soup.find_all('div', attrs={'class': 'gdReview'}):
                star1 = star.find('span', attrs={'class': 'ratingNumber mr-xsm'})
                if star1:
                    star_ratings.append(star1.text)
                else:
                    star_ratings.append('NA')

            #extracting the employee status line to extract employee status from it
            emp_line = []
            for empl in soup.find_all('div', attrs={'class': 'gdReview'}):
                empl1 = empl.find('span', attrs={'class': 'pt-xsm pt-md-0 css-1qxtz39 eg4psks0'})
                if empl1:
                    emp_line.append(empl1.text)
                else:
                    emp_line.append('NA')

            # extracting status of employee
            employee_status = []
            for i in emp_line:
                employee_status.append(i.split(',')[0])
            
            #extracting the line that contains the date and designation in the review
            date_and_des = []
            for ded in soup.find_all('div', attrs={'class': 'gdReview'}):
                ded1 = ded.find('span', attrs={'class': 'middle common__EiReviewDetailsStyle__newGrey'})
                if ded1:
                    date_and_des.append(ded1.text)
                else:
                    date_and_des.append('NA')

            #extracting the date from the date_and_des list
            date_of_review = []
            for i in date_and_des:
                date_of_review.append(i.split(' -')[0])

            #extracting the designation from the date_and_des list
            designation_of_reviewer = []
            for i in date_and_des:
                designation_of_reviewer.append(i.split('- ')[-1])

            all_pages = {}
            all_pages['Reviews'] = combined_reviews
            all_pages['Rating'] = star_ratings
            all_pages['Designation'] = designation_of_reviewer
            all_pages['Date'] = date_of_review
            all_pages['Employee Status'] = employee_status

            df_main_glassdoor = pd.DataFrame(all_pages)
            df_main_glassdoor["Source"] = "Glassdoor"
            print("\nGlassdoor's Data Extraction | Completed\n")
            return df_main_glassdoor
        
        if no_of_pages >= 1:
            # Creating final dataframe & pagination using link
            df_main_glassdoor = pd.DataFrame()
            wait = WebDriverWait(driver, 100)
            try:
                wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[4]/div/div[1]/button[2]'))).click()
            except NoSuchElementException:
                # If the first XPath is not found, try the second one
                wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[6]/div/div[1]/button[2]'))).click()
            for i in tqdm(range(1, no_of_pages + 1, 1)):
                wait = WebDriverWait(driver, 100)
                try:
                    # try:
                    #     wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[6]/div/div[1]/button[2]'))).click()
                    # except NoSuchElementException:
                    #     # If the first XPath is not found, try the second one
                    wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[6]/div/div[1]/button[2]'))).click()
                
                    time.sleep(4)
                    # Scroll to the next button element
                    next_button = driver.find_element(By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[6]/div/div[1]/button[2]')
                    actions = ActionChains(driver)
                    actions.move_to_element(next_button).perform()
                    
                    soup = BeautifulSoup(driver.page_source, 'lxml')
                    all_reviews = complete_reviews(soup)
                    df = pd.DataFrame(all_reviews)
                    df_main_glassdoor = pd.concat([df_main_glassdoor, df], ignore_index=True)
                    df_main_glassdoor = df_main_glassdoor.reset_index(drop=True)
                except:
                    print(i)



            df_main_glassdoor["Source"] = "Glassdoor"
            print("\nGlassdoor's Data Extraction | Completed\n")
            return df_main_glassdoor

def Entrackr_scrape():
    driver.get(f'https://entrackr.com/?s={s}')
    driver.maximize_window()
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    if soup.find_all('div', attrs={'class': 'elementor-column elementor-col-100 elementor-top-column elementor-element elementor-element-99d9251'}):
        soup = BeautifulSoup(driver.page_source, 'lxml')
        if "You searched for " in driver.page_source:

            def complete_reviews(soup):
                text_line = []
                for text in soup.find_all('div', attrs={'class': 'elementor-column elementor-col-100 elementor-top-column elementor-element elementor-element-99d9251'}):
                    text1 = text.find('div', attrs={'class': 'elementor-element elementor-element-0a9137c elementor-widget elementor-widget-theme-post-title elementor-page-title elementor-widget-heading'})
                    if text1:
                        text_line.append(text1.text)
                    else:
                        text_line.append('NA')

                news_text = []
                for i in text_line:
                    news_text.append(i[2:-2])
                
                date_line = []
                for date in soup.find_all('div', attrs={'class': 'elementor-column elementor-col-100 elementor-top-column elementor-element elementor-element-99d9251'}):
                    date1 = date.find('span', attrs={'class': 'elementor-icon-list-text elementor-post-info__item elementor-post-info__item--type-date'})
                    if date1:
                        date_line.append(date1.text)
                    else:
                        date_line.append('NA')

                news_date = []
                for i in date_line:
                    news_date.append(i[11:-5])

                all_pages = {}
                all_pages['Reviews'] = news_text
                all_pages['Date'] = news_date
                return all_pages

        soup = BeautifulSoup(driver.page_source, 'lxml')
        page1_news_count = soup.find_all('div', attrs={'class':'elementor-element elementor-element-0a9137c elementor-widget elementor-widget-theme-post-title elementor-page-title elementor-widget-heading'})
        

        if len(page1_news_count)<=24:
            soup = BeautifulSoup(driver.page_source, 'lxml')
            text_line = []
            for text in soup.find_all('div', attrs={'class': 'elementor-column elementor-col-100 elementor-top-column elementor-element elementor-element-99d9251'}):
                text1 = text.find('div', attrs={'class': 'elementor-element elementor-element-0a9137c elementor-widget elementor-widget-theme-post-title elementor-page-title elementor-widget-heading'})
                if text1:
                    text_line.append(text1.text)
                else:
                    text_line.append('NA')
                    
            news_text = []
            for i in text_line:
                news_text.append(i[2:-2])
            
            date_line = []
            for date in soup.find_all('div', attrs={'class': 'elementor-column elementor-col-100 elementor-top-column elementor-element elementor-element-99d9251'}):
                date1 = date.find('span', attrs={'class': 'elementor-icon-list-text elementor-post-info__item elementor-post-info__item--type-date'})
                if date1:
                    date_line.append(date1.text)
                else:
                    date_line.append('NA')

            news_date = []
            for i in date_line:
                news_date.append(i[11:-5])

            all_pages = {}
            all_pages['Reviews'] = news_text
            all_pages['Date'] = news_date
            global df_main_entrackr
            df_main_entrackr = pd.DataFrame(all_pages)
            df_main_entrackr['Source'] = 'Entrackr'
            print('\nEntrackr Data Extraction | Completed\n')
        
        elif "Next »" in driver.page_source:
                #counting the number of pages to scroll
                total_pages = driver.find_element(By.XPATH, '/html/body/div[2]/div/section/div/div/div[1]/div/div/div[2]/div/nav/a[3]').text
                total_pages = int(total_pages)
                #creating final dataframe & pagination using link
                df_main_entrackr = pd.DataFrame()
                try:
                    # Extract the reviews from the first page before proceeding to pagination
                    soup = BeautifulSoup(driver.page_source, 'lxml')
                    all_reviews = complete_reviews(soup)
                    df = pd.DataFrame(all_reviews)
                    df_main_entrackr = pd.concat([df_main_entrackr, df], ignore_index=True)
                    df_main_entrackr = df_main_entrackr.reset_index(drop=True)

                    for i in tqdm(range(2, total_pages+1)):  # Start from page 2 since page 1 data is already extracted
                        wait = WebDriverWait(driver, 100)
                        try:
                            # Find the element with the text "Next »" and click it
                            next_button = driver.find_element(By.CLASS_NAME, "page-numbers.next")
                            next_button.click()

                        except NoSuchElementException:
                            print('Next button not found')
                            break  # Exit the loop if Next button is not found

                        # Wait for the page to load
                        time.sleep(2)

                        # Extract the reviews from the current page
                        soup = BeautifulSoup(driver.page_source, 'lxml')
                        all_reviews = complete_reviews(soup)
                        df = pd.DataFrame(all_reviews)
                        df_main_entrackr = pd.concat([df_main_entrackr, df], ignore_index=True)
                        df_main_entrackr = df_main_entrackr.reset_index(drop=True)

                except Exception as e:
                    print(f"Error: {e}")
                df_main_entrackr['Source'] = 'Entrackr'
                print('\nEntrackr Data Extraction | Completed\n')
    else:
        print('No Data available on Entrackr')

searches = input("Enter company name/s(comma seperated):")
lst = searches.split(',')
opt = Options()
opt.add_argument("--incognito")
opt.add_argument('--disable-notifications')
driver = uc.Chrome(options=opt)
driver.maximize_window()
action = ActionChains(driver)

email = 'd20262054@gmail.com'
p = 'Dummysamplepassword'

email2 = 'd3280993@gmail.com'
p2 = 'Dummysamplepassword2'

GLOB_df = pd.DataFrame()

for s in lst:

    FB = Facebook_scrape()
    time.sleep(2)
    I = Indeed_scrape()
    time.sleep(2)
    # T = Twitter_scrape()
    # time.sleep(2)
    TOI = TOI_scrape()
    time.sleep(2)
    # GD = Glassdoor_scrape()
    # time.sleep(2)
    AB = AmbitionBox_scrape()
    time.sleep(2)
    EN = Entrackr_scrape()
    time.sleep(2)
    L = LinkedIn_scrape()
    time.sleep(2)

    final_df = reduce(lambda left, right: pd.merge(left,right,on=['Reviews','Date','Source'], how='outer'), [L])
    final_df = final_df.merge(AB ,how ='outer', on =['Reviews','Date','Source'])
    final_df = final_df.dropna(subset=['Reviews'])

    combined_dataframe_dhruv = pd.concat([FB,TOI,df_main_entrackr,df_main_indeed], ignore_index=True)
    combined_dataframe_dhruv = combined_dataframe_dhruv.reset_index(drop=True)

    combined_dataframe = pd.concat([final_df, combined_dataframe_dhruv], ignore_index=True)
    combined_dataframe = combined_dataframe.reset_index(drop=True)
    combined_dataframe['Date']= pd.to_datetime(combined_dataframe['Date'] , errors = 'coerce')
    combined_dataframe['Date'] = combined_dataframe['Date'].dt.date

    GLOB_df = pd.concat([GLOB_df, combined_dataframe], ignore_index=True)

driver.quit()                   #closing the  browser after extracting all the data


GLOB_df.to_csv("FINAL DATA.csv", index = False)
print('All Data Combined Together Successfully!')
print('\nNext Process: Analyzing Sentiments\n')

def sentiment_analyzer():
    print('\nRunning Sentiment Analysis\n')
    import string
    common_words = ['the','is','a','for','as','can','to','am','i','and','this','of','you','are','very','they','us','will','at','on','be','who','any','s','so','our','do']     
    def cleaning_tokenizing(review):
        if isinstance(review, float):
            return ""  # Return an empty string if the value is a float
        nopunc = [char for char in review if char not in string.punctuation]
        nopunc = ''.join(nopunc)
        nopunc = [word for word in nopunc.split() if word.lower() not in common_words]
        nopunc = ' '.join(nopunc)
        return nopunc
    
    import nltk
    from nltk.sentiment import SentimentIntensityAnalyzer
    from textblob import TextBlob
    nltk.download('vader_lexicon')
    nltk.download('punkt')
    df_main = pd.read_csv("FINAL DATA.csv")
    df_main['Text'] = df_main['Reviews'].apply(cleaning_tokenizing)
    sia = SentimentIntensityAnalyzer()

    def calc_sentiment(txt):
        if pd.isnull(txt):
            return TextBlob("").sentiment
        else:
            return TextBlob(str(txt)).sentiment
    def calc_sentiment_analyse(txt):
        if pd.isnull(txt):
            return sia.polarity_scores("")
        else:
            return sia.polarity_scores(txt)

    df_main['Sentiment'] = df_main['Text'].astype(str).apply(calc_sentiment)
    df_main['Sentiment Score'] = df_main['Text'].apply(calc_sentiment_analyse)
    df_main['Compound_score'] = None
    df_main['Compound_score_sentiment'] = None

    for i in range(0, len(df_main)): 
        df_main['Compound_score'][i] = df_main['Sentiment Score'][i]['compound']

        if (df_main['Sentiment Score'][i]['compound'] <= -0.05):
            df_main['Compound_score_sentiment'][i] = 'Negative'    
        if (df_main['Sentiment Score'][i]['compound'] >= 0.05):
            df_main['Compound_score_sentiment'][i] = 'Positive'
        if (-0.05 <= df_main['Sentiment Score'][i]['compound'] <= 0.05):
            df_main['Compound_score_sentiment'][i] = 'Neutral'

    def score(res):
        if res == 'Negative':
            return 'Bad'
        if res == 'Positive':
            return 'Good'
        if res == 'Neutral':
            return 'Neutral'

    df_main['Sentiment_score'] = df_main.Compound_score_sentiment.apply(score)

    df_main.drop("Sentiment", axis=1, inplace=True)
    df_main.drop("Sentiment Score", axis=1, inplace=True)
    df_main.drop("Compound_score_sentiment", axis=1, inplace=True)
    df_main.drop('Reviews',axis=1,inplace=True)
    df_main.rename(columns={'Compound_score': 'Sentiment Score', 'Sentiment_score': 'Sentiment','Text':'Reviews'}, inplace=True)

    df_main.to_csv('FINAL DATA ANALYSED.csv',index=False)
    print("\nAll data has been saved in file 'FINAL DATA ANALYSED' within the same location.\n")

sentiment_analyzer()

KeyboardInterrupt: 